In [1]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)

import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'  

## EEG data and foof:

In [2]:
## data loading
data = mat73.loadmat(DATA_DIR+'x_source_new.mat')  
foof = pd.read_csv(DATA_DIR+"/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/source_intercept_slope.csv")
foof

,0,1,2,3,4,5,6,7,8,9,...,127,128,129,130,131,132,133,134,135,IDs
0,0.160123,1.362617,-0.002037,1.427172,0.810964,1.391837,0.745124,1.378194,0.527491,1.433532,...,1.519565,1.004265,1.450747,0.956578,1.456952,0.611944,1.503932,0.282555,1.441946,NDARAA075AMK
1,0.988915,1.330636,1.151667,1.546990,1.479442,1.608810,1.342563,1.559859,0.460923,1.434832,...,1.461904,1.449481,1.480738,0.889122,1.282076,0.950479,1.138697,1.372408,1.505823,NDARAA112DMH
2,1.455004,1.515717,0.988018,1.367703,2.029333,1.763131,2.086803,1.763987,1.320758,1.585413,...,1.502295,1.596192,1.448922,1.180667,1.199036,1.658822,1.619147,1.182087,1.395998,NDARAA117NEJ
3,0.796931,1.547997,0.016022,0.956344,0.638462,1.222323,0.698921,1.395078,-0.130122,1.156477,...,0.998001,0.079507,1.285272,0.471817,1.175649,0.617125,1.418717,0.401675,1.222953,NDARAA947ZG5
4,-0.627481,1.123518,-0.255195,1.273727,0.197328,1.344738,0.282533,1.387126,-0.808075,1.077384,...,1.363285,0.049249,1.207528,-0.118998,1.296786,-0.253839,1.176320,-0.033039,1.318181,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,-0.124896,1.111637,0.025096,1.109243,0.393830,0.864464,0.422308,0.871363,-0.311337,0.979262,...,1.104898,0.215057,0.976700,0.769451,1.101020,-0.013650,1.085802,0.159527,1.034021,NDARRY126FA5
1408,0.259345,1.268732,0.532802,1.492400,0.722490,1.482126,0.730926,1.432505,-0.205835,1.230099,...,1.508386,0.529724,1.352627,1.061175,1.606489,0.540497,1.275614,0.835786,1.468853,NDARRY215CXQ
1409,0.396271,1.361393,0.176390,1.363488,0.753262,1.453320,0.707356,1.481902,-0.070967,1.338482,...,1.348029,0.901967,1.433662,0.392497,1.112497,0.585191,1.396128,0.307516,1.337915,NDARRY268AF2
1410,0.300712,1.325074,0.534767,1.416291,0.846201,1.590482,0.853024,1.632593,0.317269,1.416499,...,1.399847,0.863988,1.219525,1.119859,1.336477,0.667894,1.395374,0.618072,1.330237,NDARRY280KNW


In [3]:
data

{'x': array([[[2.85210744e-12, 6.84395873e-11, 1.33448613e-09, ...,
          6.40790987e-02, 4.29900039e-02, 2.21752327e-02],
         [2.16919456e-01, 2.28965785e-01, 4.42665506e-03, ...,
          7.77958670e-03, 3.80331201e-04, 5.71640022e-06],
         [2.02453249e-08, 8.55537723e-07, 2.22058385e-05, ...,
          4.21884749e-15, 0.00000000e+00, 0.00000000e+00],
         ...,
         [4.65705252e-12, 1.19924848e-10, 2.45344922e-09, ...,
          1.10354007e-02, 2.80167345e-03, 5.12511030e-04],
         [2.83217894e-13, 8.81272832e-12, 2.19235713e-10, ...,
          8.71547487e-02, 8.17854504e-02, 5.69034798e-02],
         [1.26797893e-08, 1.22521774e-07, 1.00365630e-06, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 
        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          2.83094547e-01, 2.74644661e-01, 2.61234403e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          3.17350757e-02, 1.89606919e-02, 1.06530517e-02],
  

In [4]:
# flattening
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df = np.array(df)
df = df.reshape((2042,68,79)) 
df.shape

(2042, 68, 79)

In [5]:
df = pd.DataFrame(df.reshape((df.shape[0], -1)))
df.shape


(2042, 5372)

In [6]:

type(df)
df['IDs'] = foof['IDs']
df

,0,1,2,3,4,5,6,7,8,9,...,5363,5364,5365,5366,5367,5368,5369,5370,5371,IDs
0,2.852107e-12,6.843959e-11,1.334486e-09,2.114389e-08,2.722198e-07,2.847860e-06,2.420928e-05,1.672280e-04,9.386428e-04,4.281104e-03,...,4.083400e-13,1.199041e-14,2.220446e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA075AMK
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.422859e-02,1.792809e-01,2.261597e-01,1.471350e-01,4.932961e-02,8.522938e-03,7.588586e-04,3.481949e-05,8.233298e-07,NDARAA112DMH
2,1.873727e-04,3.700203e-02,1.338341e-01,8.866054e-03,1.075761e-05,2.390714e-10,1.143530e-13,4.469758e-12,1.420740e-10,4.289156e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA117NEJ
3,4.679133e-03,2.858350e-02,1.446466e-01,3.296430e-01,3.146601e-01,1.356668e-01,1.148714e-01,3.878903e-01,4.688989e-01,1.760009e-01,...,4.721801e-02,2.446504e-02,6.717445e-02,1.737527e-01,2.703009e-01,2.480023e-01,1.340995e-01,4.273146e-02,8.024469e-03,NDARAA947ZG5
4,1.426186e-01,1.919957e-01,2.337057e-01,2.572226e-01,2.559831e-01,2.303433e-01,1.874140e-01,1.378767e-01,9.171530e-02,5.516572e-02,...,5.047329e-02,4.878144e-02,5.355981e-02,1.013861e-01,5.092052e-02,3.967086e-02,3.699821e-02,3.428801e-02,3.155638e-02,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,1.641564e-04,5.061825e-03,4.748293e-02,1.355030e-01,1.176366e-01,3.107015e-02,2.513731e-03,1.849108e-04,7.014072e-04,3.183943e-03,...,3.161835e-08,1.106568e-08,3.738061e-09,1.218730e-09,3.834910e-10,1.164628e-10,3.413536e-11,9.656276e-12,2.636114e-12,NaN
2038,1.273725e-07,2.854585e-06,4.301847e-05,4.342317e-04,2.934193e-03,1.327155e-02,4.019215e-02,8.169716e-02,1.133810e-01,1.182013e-01,...,6.180279e-12,1.819545e-12,5.200285e-13,1.443290e-13,3.885781e-14,1.021405e-14,2.553513e-15,6.661338e-16,1.110223e-16,NaN
2039,0.000000e+00,7.771561e-16,3.630429e-14,1.391887e-12,4.209189e-11,1.004152e-09,1.889601e-08,2.804845e-07,3.284092e-06,3.033118e-05,...,1.329739e-01,1.350156e-01,1.346921e-01,1.320204e-01,1.271395e-01,1.202986e-01,1.118360e-01,1.021511e-01,9.167378e-02,NaN
2040,1.363800e-08,1.521235e-07,1.428039e-06,1.127810e-05,7.492867e-05,4.187613e-04,1.968741e-03,7.785975e-03,2.590237e-02,7.248831e-02,...,5.546025e-07,2.166196e-07,8.182735e-08,2.989401e-08,1.056221e-08,3.609193e-09,1.192753e-09,3.812199e-10,1.178382e-10,NaN


## Behavioral Data:


In [7]:
behaviour_data = pd.read_csv('/home/ubuntu/Martyna/repo/AI4Health/AllData.csv')
behaviour_data = behaviour_data.rename(columns = {'EID': 'IDs'}, inplace = False)
behaviour_data.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [8]:
behaviour_data = behaviour_data[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]
behaviour_data.shape

(2579, 9)

In [9]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 9 columns (with column 'IDs').
There are 8 columns that have missing values.


,Missing Values,% of Total Values
WISC_WMI_Sum,712,27.6
WISC_VCI_Sum,702,27.2
SCARED_P_GD,576,22.3
SRS_SCI_T,448,17.4
SRS_RRB_T,447,17.3
SWAN_IN_Avg,385,14.9
SWAN_HY_Avg,385,14.9
SWAN_Avg,385,14.9


## Merge EEG and foof data together:

In [10]:
data = pd.merge(df, foof, on='IDs', how='inner')
data.shape

(2042, 5509)

## and then with the behaviorals

In [11]:
df = pd.merge(data, behaviour_data, on='IDs', how='inner')

df.shape

(1014, 5517)

## features and labels preparation:

In [12]:
#removing NaNs
df = df.dropna()
df.shape

(753, 5517)

In [13]:
df = df.drop(columns=['IDs'])
df.shape

(753, 5516)

In [14]:
df

,0,1,2,3,4,5,6,7,8,9,...,134,135,SRS_SCI_T,SRS_RRB_T,SWAN_IN_Avg,SWAN_HY_Avg,SWAN_Avg,SCARED_P_GD,WISC_WMI_Sum,WISC_VCI_Sum
3,1.426186e-01,1.919957e-01,2.337057e-01,2.572226e-01,2.559831e-01,2.303433e-01,1.874140e-01,1.378767e-01,9.171530e-02,5.516572e-02,...,-0.033039,1.318181,46.0,42.0,0.555555,0.444444,0.500000,1.0,19.0,24.0
4,3.151257e-12,8.565149e-12,2.278444e-11,5.932144e-11,1.511613e-10,3.769878e-10,9.201795e-10,2.198244e-09,5.139694e-09,1.176134e-08,...,-3.272731,0.255432,63.0,50.0,1.000000,0.777777,0.888888,1.0,17.0,18.0
6,1.028277e-05,3.372858e-05,1.036372e-04,2.980304e-04,8.017031e-04,2.016746e-03,4.743549e-03,1.043108e-02,2.144404e-02,4.121215e-02,...,0.580516,1.295198,47.0,53.0,1.777777,0.000000,0.888888,0.0,18.0,23.0
7,1.688330e-01,1.951598e-01,1.937402e-01,1.651753e-01,1.209392e-01,7.604748e-02,4.106756e-02,1.904624e-02,7.586051e-03,2.594900e-03,...,0.135288,1.029532,55.0,59.0,0.555555,0.555555,0.555555,6.0,12.0,18.0
8,7.993448e-02,1.118998e-01,1.470595e-02,1.922488e-04,4.704280e-05,1.802159e-04,6.207328e-04,1.913752e-03,5.281239e-03,1.304534e-02,...,1.342441,1.336211,60.0,53.0,2.111111,0.222222,1.166666,11.0,23.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,4.638287e-06,1.686705e-04,3.472395e-03,3.550556e-02,1.764200e-01,4.268512e-01,5.200847e-01,3.877546e-01,3.502742e-01,5.136525e-01,...,0.722683,1.396991,58.0,53.0,-1.222222,1.111111,-0.055555,1.0,19.0,31.0
1009,2.766878e-09,4.698258e-09,8.616985e-09,2.744228e-08,2.268769e-07,2.374457e-06,2.144043e-05,1.570479e-04,9.253245e-04,4.379722e-03,...,-0.367186,1.176768,69.0,60.0,-1.111111,-0.111111,-0.611111,15.0,23.0,21.0
1011,6.066814e-13,1.228982e-12,2.466000e-12,4.901385e-12,9.649892e-12,1.881950e-11,3.635581e-11,6.956935e-11,1.318688e-10,2.475973e-10,...,0.835786,1.468853,45.0,53.0,0.444444,1.000000,0.722222,5.0,13.0,14.0
1012,1.968864e-06,3.450653e-05,4.058857e-04,3.204227e-03,1.697695e-02,6.036889e-02,1.440739e-01,2.307882e-01,2.485958e-01,1.858225e-01,...,0.307516,1.337915,49.0,48.0,0.000000,0.000000,0.000000,7.0,18.0,17.0


In [15]:
X = df.iloc[:, 0:5518].to_numpy()
X.shape

(753, 5516)

In [16]:
Y = df[[ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]

## TARGETS NORMALISATION
Y = Y.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
Y_scaled = min_max_scaler.fit_transform(Y)
Y = pd.DataFrame(Y_scaled)
print(Y.shape)
Y.mean()

(753, 8)


0    0.399359
1    0.334255
2    0.616374
3    0.541390
4    0.578882
5    0.253505
6    0.451444
7    0.524610
dtype: float64

## PCA on X dimension



In [17]:
# dimensionality reduction
pca = PCA(.90) # 95% variance retained
pca.fit(X)
# transform data
X_pca = pca.transform(X)
X_pca.shape

X = X_pca
print("X:", X.shape, "Y:", Y.shape)


X: (753, 10) Y: (753, 8)


In [18]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.25)

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [19]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor



xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.5)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))

print(ytest[:10])
print(Y.mean())

Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.14
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.02
            0         1         2         3         4         5        6  \
657  0.283019  0.285714  0.722222  0.777778  0.750000  0.111111  0.50000   
565  0.283019  0.224490  0.574074  0.611111  0.592592  0.500000  0.40625   
638  0.490566  0.326531  0.833333  0.685185  0.759259  0.388889  0.18750   
419  0.075472  0.000000  0.574074  0.462963  0.518518  0.000000  0.53125   
226  0.452830  0.183673  0.592592  0.537037  0.564815  0.055556  0.50000   
144  0.132075  0.224490  0.481482  0.574074  0.527778  0.444444  0.28125   
45   0.018868  0.000000  0.648148  0.500000  0.574074  0.000000  0.37500   
81   0.415094  0.510204  0.703704  0.555555  0.629630  0.055556  0.56250   
656  0.094340  0.020408  0.074074  0.018519  0.046296  0.222222  0.81250   
635  0.320755  0.081633  0.481482  0.444445  0.462963  0.000000  0.34375   

           7  
657  0.71875  
565 

## SVR regressor

In [20]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)
# Train the regressor
mor = mor.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = mor.predict(xtest)


print("Mean squared error (SVC): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (SVC): {:.2f}".format(mor.score(xtest, ytest)))


Mean squared error (SVC): 0.02
R2 score (SVC): 0.51


## Linear Regression

In [21]:
# define model
model = LinearRegression()
# fit model
model.fit(xtrain, ytrain)

# Generate predictions for testing data
ypred = model.predict(xtest)

print("Mean squared error (LinReg): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (LinReg): {:.2f}".format(model.score(xtest, ytest)))												



Mean squared error (LinReg): 0.01
R2 score (LinReg): 0.73


## Random forest regressor

In [22]:
max_depth = 30
regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
regr_rf.fit(xtrain, ytrain)
# Predict on new data
y_rf = regr_rf.predict(xtest)
regr_rf.score(xtest, ytest)



print("Mean squared error (RandomForest): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (RandomForest): {:.2f}".format(regr_rf.score(xtest, ytest)))


Mean squared error (RandomForest): 0.01
R2 score (RandomForest): 0.61


 ## multi-output meta estimator

In [23]:
max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)
regr_multirf.fit(xtrain, ytrain)
y_multirf = regr_multirf.predict(xtest)


print("Mean squared error (regr_multirf): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (regr_multirf): {:.2f}".format(regr_multirf.score(xtest, ytest)))

Mean squared error (regr_multirf): 0.01
R2 score (regr_multirf): 0.66


## regressor chains

In [ ]:
# example of evaluating chained multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
# define base model
model = LinearSVR()
scoring = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"] 
# define the chained multioutput wrapper model
wrapper = RegressorChain(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = model_selection.cross_validate(wrapper, X, Y, scoring=scoring, cv=cv, n_jobs=-1)


In [ ]:
for i in n_scores:
    print(i)
    print('results: %.3f (%.3f)' % (mean(n_scores[i]), std(n_scores[i])))

## mlp with keras

In [28]:
model = Sequential()
model.add(Dense(100, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", metrics = ['mse'], optimizer="adam")


model.fit(xtrain, ytrain, epochs=100, batch_size=12, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score= np.array(score)
score.mean()

2021-12-04 19:44:02.809753: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-04 19:44:02.818120: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-04 19:44:02.818408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-12-04 19:44:02.818564: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-12-04 19:44:02.819707: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-12-04 19:44:02.820952: I tensorflow/stream_executor/platform/default/d

Epoch 1/100
 1/25 [>.............................] - ETA: 0s - loss: 4.7712 - mse: 4.7712

2021-12-04 19:44:03.514702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


25/25 [==============================] - 0s 10ms/step - loss: 1.5828 - mse: 1.5828 - val_loss: 0.5437 - val_mse: 0.5437
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.3936 - mse: 0.3936 - val_loss: 0.2762 - val_mse: 0.2762
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 0.2261 - mse: 0.2261 - val_loss: 0.1968 - val_mse: 0.1968
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1605 - mse: 0.1605 - val_loss: 0.1592 - val_mse: 0.1592
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1256 - mse: 0.1256 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1044 - mse: 0.1044 - val_loss: 0.1175 - val_mse: 0.1175
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0887 - mse: 0.0887 - val_loss: 0.1073 - val_mse: 0.1073
Epoch 8/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0776 - mse: 0.0776 

0.034971394